In [1]:
from __future__ import print_function
from pandas import *
import urllib2
from math import ceil
from midiutil import MIDIFile
import random
from music21 import midi as midiLib
from pygame import *
#import json

import hide_code

from ipywidgets import *
from IPython.display import display, clear_output


from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [2]:
http_handler = urllib2.HTTPHandler(debuglevel=0)
https_handler = urllib2.HTTPSHandler(debuglevel=0)
#http_proxy = urllib2.ProxyHandler({'http': 'http://proxy-chain.intel.com:911', 'https': 'http://proxy-chain.intel.com:912'})

#urllib2.ProxyHandler()

opener = urllib2.OpenerDirector()
opener.add_handler(http_handler)
opener.add_handler(https_handler)
#opener.add_handler(http_proxy)

In [3]:
def play(url):
    play = widgets.Button(description="Play")
    display(play)
    
    def on_play_clicked(b):
        response = opener.open(url)
        htmlText = response.read()
        htmlWords = htmlText.split()
        
        sentimentdf = read_csv("AFINN-111.txt", sep="\t")
        sentimentdf = sentimentdf.set_index('word')
        
        midi = []
        duration = []
        for wordu in htmlWords:
            word = wordu.lower()
            if word in sentimentdf.index:
                midi.append(sentimentdf.sentiment.loc[word]*(1+random.random()))
                duration.append(len(word))
        
        midiSer = Series(midi)
        avg = midiSer.mean()
        print('Avergage sentiment:', avg)
        
        note = float(1)/12
        avgnote = int(ceil(avg/note))
        middle = 48 
        
        degrees = []
        for i in range(len(midi)):
            p = middle + avgnote + midi[i]
            degrees.append((p, float(duration[i])/50))
            
        track    = 0
        channel  = 0
        time     = 0   # In beats
        #duration = 1   # In beats
        tempo    = 60  # In BPM
        volume   = 127 # 0-127, as per the MIDI standard

        MyMIDI = MIDIFile(1, adjust_origin=True) # One track, defaults to format 1 (tempo track automatically created)
    
        MyMIDI.addTempo(track, time, tempo)

        for pitch, duration in degrees:
            MyMIDI.addNote(track, channel, pitch, time, duration, volume)
            time = time + duration
            if time > 20:
                break
                
        with open("test.mid", "wb") as output_file:
            MyMIDI.writeFile(output_file)
            
        mf = midiLib.MidiFile()
        mf.open("test.mid")
        mf.read()
        s = midiLib.translate.midiFileToStream(mf)
        sp = midiLib.realtime.StreamPlayer(s)
        sp.play()
        
        play.close()
        
    play.on_click(on_play_clicked)
    clear_output(True)

**Instructions**

Instructions go here

Make sure the url exists and you have the http:// part

In [9]:
text = widgets.Text("http://www.foxnews.com")
display(text)
print("Enter url in box above and hit return")

def handle_submit(sender):
    play(text.value)
    print(text.value)
    clear_output(True)

text.on_submit(handle_submit)

Avergage sentiment: -0.194846269345
